In [12]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from source.read_pamap2 import read_pamap2, activityIDdict

from source.read_open_dataset import read_open_dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
train_ids = [e for e in range(100, 130)]
train_ids.remove(114)
train_ids.remove(119)
data  = read_open_dataset(seconds=4, train_ids = train_ids, test_ids=[119])

In [18]:
np.unique(data['train'][2])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [32]:
# columns = ['handAcc16_1', 'handAcc16_2', 'handAcc16_3', 'handAcc6_1', 'handAcc6_2', 'handAcc6_3', 'handGyro1', 'handGyro2', 'handGyro3', 'handMagne1', 'handMagne2', 'handMagne3']
exp_name = 'All'
# columns = ['handAcc16_1', 'handAcc16_2', 'handAcc16_3', 'handGyro1', 'handGyro2', 'handGyro3', 'heartrate']
# columns = ['heartrate']
# exp_name = 'Acc'
columns = ['handAcc16_1', 'handAcc16_2', 'handAcc16_3']
# exp_name = 'Gyr'
# columns = ['handGyro1', 'handGyro2', 'handGyro3']
data = read_pamap2(overlap=0.75, scolumns=columns)
# X_o = X

False


In [10]:
# X

In [13]:
from source.utils import idsStd
import numpy as np
# X[:, :, 0]
X = X_o


def filter_dimensions(X, all_dimensions, dimensions):
    idx = [np.where(all_dimensions == dim)[0][0] for dim in dimensions]
    return X[:, :, idx]

def folding_v2(X):
    N, T, D = X.shape
    X_f = np.zeros((N, D * T))
    for n in range(N):
        for t in range(T):
            for d in range(D):
                X_f[n, D * t + d] = X[n, t, d]
    return X_f

print(X.shape)
# X = X.transpose([0, 2, 1])
print(X.shape)

all_ids = np.unique(I)
ind_std = idsStd(all_ids, X, I)

Z_SCORE_NORM = True     

if Z_SCORE_NORM:
    N_tr, T, D = X.shape
    X_sh = np.zeros(X.shape)
    for i in range(N_tr):
        for k in range(D):
            mag = np.mean(X[i, :, k], axis = 0)
            indice = np.where(all_ids ==  I[i])[0][0]
            std = ind_std[indice][k] * 6
            X_sh[i, :, k] = (X[i, :, k] - mag) / std
            # X_train_sh[i, :, k] = np.concatenate([[0], fft(X_train[i, :, k])[1:]])
X = X_sh

print(X.shape)
print(y.shape)
print(I.shape)
N, D, T = X.shape
# for n in range(N):
#     for d in range(D):
#         X[n, d, :] = X[n, d, :] - X[n, d, :].mean()
        
features = folding_v2(X.transpose([0, 2, 1]))
print(features.shape)
# print(X[:, :, 0].flatten().shape)
# print(np.count_nonzero(np.isnan(X[:, :, 0])))


(13956, 400, 6)
(13956, 400, 6)
(13956, 400, 6)
(13956,)
(13956,)
(13956, 2400)


In [14]:
import umap

reducer = umap.UMAP(n_components=2, metric='braycurtis')
coords = reducer.fit_transform(features, y=y)

In [15]:
import sys
sys.path.insert(0, '/home/texs/Documentos/Repositories/mts_viz')

from server.source.storage import MTSStorage

In [17]:
storage = MTSStorage('pamap2')
storage.load()

# activities_map = {act: str(act) for act in np.unique(y)}
storage.add_mts(
    exp_name, X, 
    columns, 
    coords={'umap': coords}, 
    labels={'activities': y}, 
    labelsNames={'activities': activityIDdict},
    sampling = True,
    n_samples = 2000
)
storage.save()